Engy-4350: Nuclear Reactor Engineering Spring 2019 UMass Lowell; Prof. V. F. de Almeida **03Feb2019**

# 02. Nuclear Data and Data Processing 
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

---
## Table of Contents
* [Objectives](#obj)
* [Introduction](#intro)
* [Fission fragment yield](#ffy1)
* [Beta decay example](#beta1)
* [Cross section example](#xs1)
---

## Objectives<a id="obj"></a>
+ Demonstrate how to obtain traceable nuclear data and have them available through the notebook for analysis and problem solving.

## Introduction<a id="intro"></a>

Many nuclear data sets are availale through the US [National Nuclear Data Center](https://www.nndc.bnl.gov/). Some important sites are as follows:

+ [Interactive chart of nuclides](https://www.nndc.bnl.gov/nudat2/).
+ [Nuclide energy level and $\gamma$ emission](https://www.nndc.bnl.gov/nudat2/indx_adopted.jsp).
+ [Nuclear wallet card](https://www.nndc.bnl.gov/nudat2/indx_sigma.jsp).
+ [Decay radiation search](https://www.nndc.bnl.gov/nudat2/indx_dec.jsp)
+ [Q-value calculator](https://www.nndc.bnl.gov/qcalc/) (difference of kinetic energy of products and reactant particles in a nuclear reaction, *i.e.* energy release).
+ [Atomic mass evaluator](http://amdc.impcas.ac.cn/web/masseval.html).
+ [Evaluated nuclear data file (ENDF)](https://www.nndc.bnl.gov/exfor/endf00.jsp).
+ [Cross sections](https://www.nndc.bnl.gov/sigma/index.jsp) for many nuclear reactions.

## Fission fragment yield<a id="ffy1"></a>

+ Using the [ENDF](https://www.nndc.bnl.gov/exfor/endf00.jsp) help tab, look for an existing example. In this case follow the `Example 9: 235U, individual and cumulative fission product yields, US library, numerical values`. 
+ Use the *extended retrieval tab* and provide the input: `Target=u-235, Sub-library (Projectile) = *fpy* and Library=ENDF/B-VII.0`. 
+ In the new window, select the library and click on `FPY(A)`. 
+ Select the thermal neutron fission product yield data `FPYI,U-235,ENDF/B-VIII.0,Ei=0.0253eV` and repaint the plot.
+ Click on `See: plotted data`.
+ Use the browser save option to create a file with the raw data.


In [ ]:
'''View the raw data'''
!cat data/u-235-fpy-thermal.dat

In [ ]:
'''Visualize the data'''
# Pandas: python package for tabular data analysis
import pandas as pd

# build a "data frame (i.e. table)"
df = pd.read_csv('data/u-235-fpy-thermal.dat', 
                  names=['A','FP Yield','Uncertainty'], 
                  skiprows=11,
                  delim_whitespace=True)
#print(df)
import matplotlib.pyplot as plt
plt.figure(1, figsize=(8, 6))
 
plt.errorbar(df['A'], df['FP Yield'], yerr=df['Uncertainty'],
             fmt='ok',ecolor='r',label='experimental (ENDF/B-VIII.0)',capsize=3)
    
plt.xlabel(r'A',fontsize=18)
plt.ylabel(r'Fission Product Yield',fontsize=18)
plt.title('Fission of $^{235}_{92}$U (E=0.0253 eV)',fontsize=20)
plt.legend(loc='best',fontsize=12)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
    
plt.grid(True)
plt.show()
print('')

## Generalized least-squares fitting with Fourier basis functions<a id="intro"></a>

The least-squares method with Fourier basis functions is a powerful computational tool for data fitting and data analysis, the course notes OneNote [ChEn-3170-gen-lsq](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/Ep9qLSMssi1MjWvl2wYlTHkBSd5aUUoo1fIoe5pswIV0vA?e=Mhkd6f) collects elements of the theory (see also  [ChEn-3170](https://github.com/dpploy/chen-3170) Notebook 12). 

The Fourier expansion for approximating the fission yield $y(A)$ is

\begin{equation*}
y(A) = \sum\limits_{k=0}^N \alpha_k\,\cos(k\,\mu\,A) + \beta_k\,\sin(k\,\mu\,A)
\end{equation*}

If we have a set of values of the independent variable $A_i, i=1,\ldots,m$, the above Fourier expression when applied to every $A_i$ gives

where $\Amtrx =  \begin{pmatrix}
1 & \cos(\mu\,t_1) & \sin(\mu\,t_1) & \cos(2\mu\,t_1) & \sin(2\mu\,t_1) & \ldots & \cos(N\mu\,t_1) & \sin(N\mu\,t_1)  \\
1 & \cos(\mu\,t_2) & \sin(\mu\,t_2) & \cos(2\mu\,t_2) & \sin(2\mu\,t_2) & \ldots & \cos(N\mu\,t_2) & \sin(N\mu\,t_2)  \\
\vdots  & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
1 & \cos(\mu\,t_m) & \sin(\mu\,t_m) & \cos(2\mu\,t_m) & \sin(2\mu\,t_m) & \ldots & \cos(N\mu\,t_m) & \sin(N\mu\,t_m)  \\
 \end{pmatrix}$, 
 $\xvec =  \begin{pmatrix}
  \alpha_0 \\ 
  \alpha_1 \\
  \beta_1 \\
  \vdots \\
  \alpha_N \\
  \beta_N \\ 
 \end{pmatrix}$, 
and 
$\bvec = \begin{pmatrix}
 y_1 \\ 
 y_2 \\ 
 \vdots  \\ 
 y_m \\ 
\end{pmatrix} $.

In [ ]:
import math
import numpy as np

period = np.max(df['A']) - np.min(df['A'])  # [h]
omega  = 1/period          # cycle frequency [1/h]
mu     = 2*math.pi * omega # radian frequency [rad/h]

n_pairs = 6

def build_fourier_matrix( mu, n_pairs, abscissa_vec ):

    import numpy as np
    a_mtrx = np.ones((abscissa_vec.size, 2*n_pairs + 1))

    for k in range(n_pairs+1):
        if k == 0:
            continue
        a_mtrx[:,2*k-1] = np.cos(k * mu * abscissa_vec)
        a_mtrx[:,2*k]   = np.sin(k * mu * abscissa_vec)

    return a_mtrx            

a_mtrx = build_fourier_matrix( mu, n_pairs, np.array(df['A']) )

b_vec = np.array(df['FP Yield'])

rank = np.linalg.matrix_rank(a_mtrx)
#print('rank(A) =',rank)
assert rank == 2*n_pairs+1

In [ ]:
def build_fourier_matrix( mu, n_pairs, abscissa_vec ):

    import numpy as np
    a_mtrx = np.ones((abscissa_vec.size, 2*n_pairs + 1))

    for k in range(n_pairs+1):
        if k == 0:
            continue
        a_mtrx[:,2*k-1] = np.cos(k * mu * abscissa_vec)
        a_mtrx[:,2*k]   = np.sin(k * mu * abscissa_vec)

    return a_mtrx            

In [ ]:
a_mtrx = build_fourier_matrix( mu, n_pairs, np.array(df['A']) )

b_vec = np.array(df['FP Yield'])

rank = np.linalg.matrix_rank(a_mtrx)
#print('rank(A) =',rank)
assert rank == 2*n_pairs+1

In [ ]:
x_vec = np.linalg.solve( a_mtrx.transpose()@a_mtrx, a_mtrx.transpose()@b_vec )

In [ ]:
'''Function: plot the LS Fourier fit and all modes'''

def plot_fourier_fit(mass_number_expt, fp_yield_expt, mu, x_vec):
    
    import matplotlib.pyplot as plt
    
    plt.figure(2, figsize=(7,7))

    # plot experimental data
    plt.plot(mass_number_expt, fp_yield_expt,'r*',label='experimental')

    # plot LS Fourier fitting
    n_plot_pts = 100
    mass_number_plot = np.linspace(mass_number_expt[0],mass_number_expt[-1],n_plot_pts)
    a_mtrx = build_fourier_matrix (mu, n_pairs, mass_number_plot )
    fp_yield_plot = a_mtrx @ x_vec
    
    plt.plot( mass_number_plot, fp_yield_plot,'b--',label='LS Fourier fitting' )

    from engy_4350.help import color_map
    colors = color_map(a_mtrx.shape[1])
    
    for j in range(a_mtrx.shape[1]):

        if j != 0:
            color=colors[j-1]
        
        if j == 0:
            color='black'
            k = 0
            label=r'$\alpha_{%i}$=%4.2e'%(k,x_vec[j])
        elif j%2 == 0:
            k = j/2
            label=r'$\beta_{%i}$(=%4.2e) sin($%i\mu t$)'%(k,x_vec[j],k)
        else:
            k = (j+1)/2
            label=r'$\alpha_{%i}$(=%4.2e) cos($%i\mu t$)'%(k,x_vec[j],k)
        
        vertical_offset = 0.03 # to improve visibility of modes
        if j == 0:
            vertical_offset = 0
        
        plt.plot(mass_number_plot, x_vec[j]*a_mtrx[:,j]-vertical_offset,label=label,color=color )
        
    plt.xlabel(r'A',fontsize=18)
    plt.ylabel(r'FP Yield',fontsize=18)
    plt.title('Fission of $^{235}_{92}$U (E=0.0253 eV)',fontsize=20)

    (x_min,x_max) = plt.xlim()
    dx = abs(x_max-x_min)
    x_text = x_min + dx*0.25
    
    (y_min,y_max) = plt.ylim()
    dy = abs(y_max-y_min)
    y_text = y_min + dy*0.02
    
    plt.text(x_text, y_text, 
             r'$\mu=$%8.2e [rad/h],   $N$=%i'%
             (mu,n_pairs),fontsize=16)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(loc='upper right',bbox_to_anchor=(1.55, 1),fontsize=12)
    plt.grid(True)
    plt.show()
    print('')
    return

In [ ]:
'''Plot the LS Fourier fit'''

plot_fourier_fit(np.array(df['A']), np.array(df['FP Yield']), mu, x_vec)

In [ ]:
def fp_yield( mass_number ):
    '''
    Fission product yields curve fitting.

    Parameters
    ----------
    mass_number: float or numpy.ndarray, required
        Nuclide mass number. If `mass_number` is an array, 
        the return value is also an array of values.
    x_a_0: float, required
        Mole fraction of species A.
    
    Returns
    -------
    value: float or numpy.ndarray
        Value or vector of values of the fission product yield
        evaluated at `mass_number`.

    Examples
    --------
    '''
    assert np.min(mass_number) >= min(df['A']) and np.max(mass_number) <= max(df['A']),'Out of range.'
    
    a_mtrx = build_fourier_matrix (mu, n_pairs, mass_number )
    fp_yield = a_mtrx @ x_vec
    
    fp_yield[fp_yield<0] = 0.0 # correct for negative values in the curve fitting
        
    return fp_yield

In [ ]:
'''Usage of function'''
np.set_printoptions(precision=5)
# single value A
print('y(80) = ',fp_yield(np.array([80])))

# multiple values of A at once
print('y([80,85,90,120,150]) = ',fp_yield(np.array([80,90,100,110,120,130,140,150,160])))

In [ ]:
def build_fourier_matrix_prime( mu, n_pairs, abscissa_vec ):

    import numpy as np
    a_mtrx = np.ones((abscissa_vec.size, 2*n_pairs + 1))

    for k in range(n_pairs+1):
        if k == 0:
            continue
        a_mtrx[:,2*k-1] = -np.sin(k * mu * abscissa_vec)
        a_mtrx[:,2*k]   =  np.cos(k * mu * abscissa_vec)

    return a_mtrx  

In [ ]:
def fp_yield_prime( mass_number ):
    '''
    Fission product yields curve fitting.

    Parameters
    ----------
    mass_number: float or numpy.ndarray, required
        Nuclide mass number. If `mass_number` is an array, 
        the return value is also an array of values.
    x_a_0: float, required
        Mole fraction of species A.
    
    Returns
    -------
    value: float or numpy.ndarray
        Value or vector of values of the fission product yield
        evaluated at `mass_number`.

    Examples
    --------
    '''
    assert np.min(mass_number) >= min(df['A']) and np.max(mass_number) <= max(df['A']),'Out of range.'
    
    a_mtrx = build_fourier_matrix_prime(mu, n_pairs, mass_number )
    fp_yield_prime = a_mtrx @ x_vec
        
    return fp_yield_prime

In [ ]:
'''Usage of function'''
np.set_printoptions(precision=5)
# single value A
print("y'(80) = ",fp_yield_prime(np.array([80])))

# multiple values of A at once
print("y'([80,85,90,120,150]) = ",fp_yield_prime(np.array([80,90,100,110,120,130,140,150,160])))

In [ ]:
"""Newton's method"""

def newton_solve( mass_num_0=0.0, k_max=30, tolerance=1.0e-10, verbose=True ):

    # Other initialization
    delta_k = 1e+10
    f_k     = 1e+10
    mass_num = mass_num_0

    if verbose is True:
        print('\n')
        print('******************************************************')
        print("          Newton's Method Iterations                  ")
        print('******************************************************')
        print("k |  f(a_k)  |  f'(a_k) | |del a_k| |    a_k   |convg|")
        print('------------------------------------------------------')

    import math
    k = 0
    
    while (abs(delta_k) > tolerance or abs(f_k) > tolerance) and k <= k_max:
        
        f_k       = fp_yield( mass_num )
        f_prime_k = fp_yield_prime( mass_num )
        
        delta_k_old = delta_k
        delta_k = -f_k / f_prime_k
        
        mass_num += delta_k
        
        if k > 0:
            if delta_k != 0.0 and delta_k_old != 0.0:
                convergence_factor = math.log(abs(delta_k),10) / math.log(abs(delta_k_old),10)
            else:
                convergence_factor = 0.0  
        else:
            convergence_factor = 0.0
            
        k = k + 1
        
        if verbose is True:
            print('%2i %+5.3e %+5.3e %+5.3e  %+5.3e %5.2f'%\
                  (k,f_k,f_prime_k,abs(delta_k),mass_num,convergence_factor))

    if verbose is True:
        print('******************************************************') 
        print('Root = %8.5e'%mass_num)
    
    return mass_num

In [ ]:
'''Find root of y(A)'''

mass_num_0 = np.array([90])
k_max = 20
tolerance = 1.0e-8

mass_num = newton_solve( mass_num_0,k_max,tolerance)


print('')
print('Root')
print('A = %5.3e\n'%mass_num)

## Beta decay example<a id="beta1"></a>





## Cross section example<a id="xs1"></a>
Using the [cross section data site](https://www.nndc.bnl.gov/sigma/index.jsp) click on U in the periodic table and choose $^{235}_{92}$U from the isotope list. Select the neutron capture reaction cross section $^{235}_{92}$U(n,$\gamma$) and plot the data. Choose `view evaluated data` and click on the `Text` link to view the data in a two-column arrangement (energy versus cross section). Using the browser, save data into a text file (`u-235-sigma-c.dat`). Saving the file in a directory `data/` relative to this notebook, the data can be imported as follows.

In [ ]:
'''View the raw data'''
!cat data/u-235-sigma-c.dat

In [ ]:
'''Visualize the data'''

# Pandas: python package for tabular data analysis
import pandas as pd

df = pd.read_csv('data/u-235-sigma-c.dat', 
                  names=['energy [eV]','sigma_c [b]'], 
                  skiprows=3)

#print(df)
ax = df.plot(loglog=True, x='energy [eV]', y='sigma_c [b]',legend=False,
             title='Capture $\sigma_c$ for $^{235}_{92}$U(n,$\gamma$)')
ax.set(ylabel='$\sigma_c$ [b]')
ax.grid()
#plt.show()